In [3]:
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit_optimization.algorithms import GroverOptimizer, MinimumEigenOptimizer
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp
import numpy as np
from docplex.mp.model import Model

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [4]:
def checkCol(arr,i,j,numOfElementsinrow):
    if abs(i-j) % numOfElementsinrow ==0:
        arr[i][j]=1
def checkRow(arr,i,j,numOfElementsInRows):
    if (i // numOfElementsInRows) == (j // numOfElementsInRows):
        arr[i][j] = 1
def genMat(n,numOfElementsInRow):
    mat = []
    for i in range(n):
        mat.append( [0] * n)
    for i in range(len(mat)):
        for j in range(len(mat[i])):
            if i!=j:
                checkCol(mat,i,j,numOfElementsInRow)
                checkRow(mat,i,j,numOfElementsInRow)
    print(mat)
genMat(4,2)

[[0, 1, 1, 0], [1, 0, 0, 1], [1, 0, 0, 1], [0, 1, 1, 0]]


In [5]:
c=[[100,1, 30],
   [20,90, 5],
   [25,80, 95]]
p=[2,2,4]
alpha_matrix=[[c[i][j]*p[j] for j in range(len(p))] for i in range(len(c))] 
alpha_matrix
alpha_one=np.array(alpha_matrix)
alpha_one

array([[200,   2, 120],
       [ 40, 180,  20],
       [ 50, 160, 380]])

In [6]:
alpha=np.reshape(alpha_one,len(alpha_one)*len(alpha_one[0]))
alpha

array([200,   2, 120,  40, 180,  20,  50, 160, 380])

In [7]:
mod = QuadraticProgram("QUBO")
mod.export_as_lp_string()

'\\ This file has been generated by DOcplex\n\\ ENCODING=ISO-8859-1\n\\Problem name: QUBO\n\nMinimize\n obj:\nSubject To\n\nBounds\nEnd\n'

In [8]:
k=mod.binary_var_list(len(alpha))

In [12]:
def checkCol(arr,i,j,numOfElementsinrow):
    if abs(i-j) % numOfElementsinrow ==0:
        arr[i][j]=1
def checkRow(arr,i,j,numOfElementsInRows):
    if (i // numOfElementsInRows) == (j // numOfElementsInRows):
        arr[i][j] = 1
def genMat(n,numOfElementsInRow):
    mat = []
    for i in range(n):
        mat.append( [0] * n)
    for i in range(len(mat)):
        for j in range(len(mat[i])):
            if i!=j:
                checkCol(mat,i,j,numOfElementsInRow)
                checkRow(mat,i,j,numOfElementsInRow)
    return mat
quad=np.array(genMat(len(alpha),3))

In [13]:
quad[quad>0]=-100
quad

array([[   0, -100, -100, -100,    0,    0, -100,    0,    0],
       [-100,    0, -100,    0, -100,    0,    0, -100,    0],
       [-100, -100,    0,    0,    0, -100,    0,    0, -100],
       [-100,    0,    0,    0, -100, -100, -100,    0,    0],
       [   0, -100,    0, -100,    0, -100,    0, -100,    0],
       [   0,    0, -100, -100, -100,    0,    0,    0, -100],
       [-100,    0,    0, -100,    0,    0,    0, -100, -100],
       [   0, -100,    0,    0, -100,    0, -100,    0, -100],
       [   0,    0, -100,    0,    0, -100, -100, -100,    0]])

In [14]:

mod.maximize(linear=alpha,quadratic=quad)
mod.export_as_lp_string()

'\\ This file has been generated by DOcplex\n\\ ENCODING=ISO-8859-1\n\\Problem name: QUBO\n\nMaximize\n obj: 200 x0 + 2 x1 + 120 x2 + 40 x3 + 180 x4 + 20 x5 + 50 x6 + 160 x7 + 380 x8\n      + [ - 400 x0*x1 - 400 x0*x2 - 400 x0*x3 - 400 x0*x6 - 400 x1*x2\n      - 400 x1*x4 - 400 x1*x7 - 400 x2*x5 - 400 x2*x8 - 400 x3*x4 - 400 x3*x5\n      - 400 x3*x6 - 400 x4*x5 - 400 x4*x7 - 400 x5*x8 - 400 x6*x7 - 400 x6*x8\n      - 400 x7*x8 ]/2\nSubject To\n\nBounds\n 0 <= x0 <= 1\n 0 <= x1 <= 1\n 0 <= x2 <= 1\n 0 <= x3 <= 1\n 0 <= x4 <= 1\n 0 <= x5 <= 1\n 0 <= x6 <= 1\n 0 <= x7 <= 1\n 0 <= x8 <= 1\n\nBinaries\n x0 x1 x2 x3 x4 x5 x6 x7 x8\nEnd\n'

In [15]:
print("constant:\t\t\t", mod.objective.constant)
print("linear dict:\t\t\t", mod.objective.linear.to_dict())
print("linear array:\t\t\t", mod.objective.linear.to_array())
print("linear array as sparse matrix:\n", mod.objective.linear.coefficients, "\n")
print("quadratic dict w/ index:\t", mod.objective.quadratic.to_dict())
print("quadratic dict w/ name:\t\t", mod.objective.quadratic.to_dict(use_name=True))
print(
    "symmetric quadratic dict w/ name:\t",
    mod.objective.quadratic.to_dict(use_name=True, symmetric=True),
)
print("quadratic matrix:\n", mod.objective.quadratic.to_array(), "\n")
print("symmetric quadratic matrix:\n", mod.objective.quadratic.to_array(symmetric=True), "\n")
print("quadratic matrix as sparse matrix:\n", mod.objective.quadratic.coefficients)

constant:			 0.0
linear dict:			 {0: 200, 1: 2, 2: 120, 3: 40, 4: 180, 5: 20, 6: 50, 7: 160, 8: 380}
linear array:			 [200   2 120  40 180  20  50 160 380]
linear array as sparse matrix:
   (0, 0)	200
  (0, 1)	2
  (0, 2)	120
  (0, 3)	40
  (0, 4)	180
  (0, 5)	20
  (0, 6)	50
  (0, 7)	160
  (0, 8)	380 

quadratic dict w/ index:	 {(0, 1): -200, (0, 2): -200, (1, 2): -200, (0, 3): -200, (1, 4): -200, (3, 4): -200, (2, 5): -200, (3, 5): -200, (4, 5): -200, (0, 6): -200, (3, 6): -200, (1, 7): -200, (4, 7): -200, (6, 7): -200, (2, 8): -200, (5, 8): -200, (6, 8): -200, (7, 8): -200}
quadratic dict w/ name:		 {('x0', 'x1'): -200, ('x0', 'x2'): -200, ('x1', 'x2'): -200, ('x0', 'x3'): -200, ('x1', 'x4'): -200, ('x3', 'x4'): -200, ('x2', 'x5'): -200, ('x3', 'x5'): -200, ('x4', 'x5'): -200, ('x0', 'x6'): -200, ('x3', 'x6'): -200, ('x1', 'x7'): -200, ('x4', 'x7'): -200, ('x6', 'x7'): -200, ('x2', 'x8'): -200, ('x5', 'x8'): -200, ('x6', 'x8'): -200, ('x7', 'x8'): -200}
symmetric quadratic dict w/ name

In [16]:
from qiskit import IBMQ
# IBMQ.save_account(TOKEN)
IBMQ.load_account() # Load account from disk
IBMQ.providers()    # List all available providers
provider = IBMQ.get_provider(hub='ibm-q')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_bogota') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_extended_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_m

In [17]:

backend = provider.get_backend('simulator_statevector')
backend

<IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q', group='open', project='main')>

In [23]:
grover_optimizer = GroverOptimizer(12,num_iterations=12, quantum_instance=backend)
results = grover_optimizer.solve(mod)
print("x={}".format(results.x))
print("fval={}".format(results.fval))

x=[1. 0. 0. 0. 0. 0. 0. 0. 1.]
fval=580.0
